In [ ]:
import pandas as pd
from google.colab import drive
import numpy as np
from datetime import datetime

drive.mount('/content/drive')

path = '/content/drive/MyDrive/Evictions.csv'

import pandas as pd
df = pd.read_csv(path)
df

Mounted at /content/drive


,Court Index Number,Docket Number,Eviction Address,Eviction Apartment Number,Executed Date,Marshal First Name,Marshal Last Name,Residential/Commercial,BOROUGH,Eviction Postcode,Ejectment,Eviction/Legal Possession,Latitude,Longitude,Community Board,Council District,Census Tract,BIN,BBL,NTA
0,50562/18,204524,367 95TH STREET,3F,03/28/2018,Richard,Capuano,Residential,BROOKLYN,11209,Not an Ejectment,Possession,40.616466,-74.032064,10.0,43.0,54.0,3155501.0,3.061140e+09,Bay Ridge
1,318425/23,35977,1106 AVENUE K,3X,09/11/2024,Edward,Guida,Residential,BROOKLYN,11230,Not an Ejectment,Possession,40.622388,-73.964525,14.0,44.0,534.0,3180189.0,3.067220e+09,Midwood
2,46726/17,338972,1121 ELDER AVENUE,D10,01/08/2018,Thomas,Bia,Residential,BRONX,10472,Not an Ejectment,Possession,40.826776,-73.878762,9.0,17.0,5002.0,2023838.0,2.037390e+09,Soundview-Bruckner
3,301288/22,13057,34-05 108TH STREET,3A,04/29/2024,David,Smith,Residential,QUEENS,11368,Not an Ejectment,Possession,40.756109,-73.860674,3.0,21.0,381.0,4043565.0,4.017500e+09,North Corona
4,35954/15,312984,2720 DECATUR AVENUE,4,06/05/2017,Thomas,Bia,Residential,BRONX,10458,Not an Ejectment,Possession,40.865294,-73.888287,7.0,15.0,40502.0,2016378.0,2.032780e+09,Bedford Park-Fordham North
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
110380,B302679/22,124257,158 E. 184 STREET,2D,01/15/2025,Darlene,Barone,Residential,BRONX,10468,Not an Ejectment,Possession,40.859515,-73.899471,5.0,14.0,23703.0,2013784.0,2.031640e+09,Fordham South
110381,0311271/22,7200,18-36 130TH STREET,2,04/24/2024,David,Smith,Residential,QUEENS,11356,Not an Ejectment,Possession,40.782959,-73.838645,7.0,19.0,947.0,4099007.0,4.041350e+09,College Point
110382,76114/17,155546,146-58 181 STREET (APT.NO.BASEMENT),NaN,09/06/2018,Ronald,Pazant,Residential,QUEENS,11413,Not an Ejectment,Possession,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
110383,56172/17NY,62746,398 WEST STREET,GROUND FLOOR STORE,04/25/2017,Justin,Grossman,Commercial,MANHATTAN,10014,Not an Ejectment,Possession,40.733370,-74.010251,2.0,3.0,75.0,1011988.0,1.006360e+09,West Village


In [ ]:
import pandas as pd
import re
import numpy as np

def find_address_column(df):
    address_pattern = re.compile(r'\b\d+\s+\w+\s+(?:way|place|drive|lane|street|avenue|road|boulevard|wy|pl|dr|ln|st|ave|rd|blvd||Wy|Pl|St|Ave|Rd|Blvd|Dr|Ln|DRIVE|WAY|PLACE|LANE|STREET|AVENUE|BOULEVARD|ROAD|ST|DR|LN|WY|PL|AVE|RD|Way|Place|Drive|Lane|Boulevard|Street|Avenue|Road)\b', re.IGNORECASE)
    for column in df.columns:
        if df[column].astype(str).str.contains(address_pattern).any():
            return column
    return None


address_column = find_address_column(df)
if address_column:
    print(f"Address column found: {address_column}")
else:
    print("No address column found.")

Address column found: Eviction Address


In [ ]:
import pandas as pd
import re

# Dictionary mapping abbreviations to full names
abbreviation_map = {
    "St": "Street",
    "Ave": "Avenue",
    "Rd": "Road",
    "Blvd": "Boulevard",
    "Dr": "Drive",
    "Ln": "Lane",
    "Wy": "Way",
    "Pl": "Place"
}

# Function to replace abbreviations safely using regex with word boundaries
def expand_abbreviations(address):
    for abbr, full in abbreviation_map.items():
        address = re.sub(rf'\b{abbr}\b', full, address, flags=re.IGNORECASE)  # Replace full words only
    return address

# Apply function **only to first 10 rows**
df.loc[:9, address_column] = df.loc[:9, address_column].astype(str).apply(expand_abbreviations)

# Print results
print(df.head(10))  # Show first 10 rows to verify

  Court Index Number  Docket Number                       Eviction Address  \
0           50562/18          204524                       367 95TH STREET   
1          318425/23           35977                         1106 AVENUE K   
2           46726/17          338972                     1121 ELDER AVENUE   
3          301288/22           13057                    34-05 108TH STREET   
4           35954/15          312984                   2720 DECATUR AVENUE   
5         B021796/18          394014             280-300 EAST 161ST STREET   
6          302560/23           80118                   152 WOODVALE AVENUE   
7         R 50575/17           68636  150 VAN PELT AVENUE  ENTIRE PREMISES   
8           50438/19           77888       36 WALKER DRIVE ENTIRE PREMISES   
9           50289/19           27357                  1253 HERKIMER STREET   

  Eviction Apartment Number Executed Date Marshal First Name  \
0                        3F    03/28/2018            Richard   
1            

In [ ]:
import pandas as pd
import re

# Dictionary mapping full street names to abbreviations
full_to_abbreviation = {
    "Street": "St",
    "Avenue": "Ave",
    "Road": "Rd",
    "Boulevard": "Blvd",
    "Drive": "Dr",
    "Lane": "Ln",
    "Way": "Wy",
    "Place": "Pl"
}

# Function to abbreviate full street names (case-insensitive)
def abbreviate_address(address):
    for full_name, abbr in full_to_abbreviation.items():
        address = re.sub(rf'\b{full_name}\b', abbr, address, flags=re.IGNORECASE)  # Case-insensitive replacement
    return address

# Apply function **only to the last 10 rows**
df.loc[df.index[-10:], address_column] = df.loc[df.index[-10:], address_column].astype(str).apply(abbreviate_address)

# Print results
print(df.tail(10))  # Show last 10 rows to verify

       Court Index Number  Docket Number                  Eviction Address  \
110375           78112/16           67411                   1955 FIRST Ave   
110376           54704/17           10049                    34-32 43RD ST   
110377          337024/23          130571                1240 MORRISON Ave   
110378           65294/19           11660                     98 LEWIS Ave   
110379           B1404/22          122417                   726 E 134TH St   
110380         B302679/22          124257                    158 E. 184 St   
110381         0311271/22            7200                   18-36 130TH St   
110382           76114/17          155546  146-58 181 St (APT.NO.BASEMENT)   
110383         56172/17NY           62746                      398 WEST St   
110384          K61059/17           78743                 8645 BAY PARKWAY   

       Eviction Apartment Number Executed Date Marshal First Name  \
110375                       617    08/02/2017              Henry   
110

In [ ]:
import pandas as pd
import re

# Dictionary mapping full street names to abbreviations
full_to_abbreviation = {
    "Street": "St",
    "Avenue": "Ave",
    "Road": "Rd",
    "Boulevard": "Blvd",
    "Drive": "Dr",
    "Lane": "Ln",
    "Way": "Wy",
    "Place": "Pl"
}

# Function to abbreviate full street names (case-insensitive)
def abbreviate_address(address):
    for full_name, abbr in full_to_abbreviation.items():
        address = re.sub(rf'\b{full_name}\b', abbr, address, flags=re.IGNORECASE)  # Case-insensitive replacement
    return address

# Apply function to the entire address column
df[address_column] = df[address_column].astype(str).apply(abbreviate_address)

# Print results
print(df[address_column].head(10))  # Show first 10 updated addresses

0                          367 95TH St
1                           1106 Ave K
2                       1121 ELDER Ave
3                       34-05 108TH St
4                     2720 DECATUR Ave
5                280-300 EAST 161ST St
6                     152 WOODVALE Ave
7    150 VAN PELT Ave  ENTIRE PREMISES
8         36 WALKER Dr ENTIRE PREMISES
9                     1253 HERKIMER St
Name: Eviction Address, dtype: object


In [ ]:
full_street_rows = df[df[address_column].str.contains('|'.join(full_to_abbreviation.keys()), case=False, regex=True)]
print(f"✅ After reprocessing, the column now has an abbreviated street address format. However, there are {len(full_street_rows)} addresses that are not updating due to typos, spacing, and unrecognizeable characters such as quotations, symbols etc. Consider updating these anomalies before using data set.")
print(f"If there are anomolies to the address column, examples will appear below:")
print(full_street_rows[[address_column]].head(10))  # Show sample of unchanged rows

✅ After reprocessing, the column now has an abbreviated street address format. However, there are 3277 addresses that are not updating due to typos, spacing, and unrecognizeable characters such as quotations, symbols etc. Consider updating these anomalies before using data set.
If there are anomolies to the address column, examples will appear below:
                                Eviction Address
197  4131-4139 BROADWAY A /K/A 700 WEST 175TH St
215                                2724 BROADWAY
220                                3750 BROADWAY
235                        5008 BROADWAY APT. 4H
266                                 861 BROADWAY
338                 111 E MOSHOLU  PARKWAY NORTH
362                     8602 FT.HAMILTON PARKWAY
375                                 180 BROAD St
422                            1450 GATEWAY Blvd
428                    1881 SHORE PARKWAY 3RD FL
